# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
env = UnityEnvironment(file_name='/home/straight/Programming/JupyterNotebook/DRL nanodegree/3. Policy-Based methods/p2_continuous-control/Reacher_Linux/Reacher.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.12799999713897706


### 3. DDPG agent

In [6]:
import random
import torch
from collections import deque
import matplotlib.pyplot as plt
import time
%matplotlib inline

from ddpg_agent import Agent

In [7]:
# load the agent with DDPG algorithm
agent = Agent(state_size=state_size, action_size=action_size, n_agents = num_agents, random_seed=2)

In [8]:
## test
state = env_info.vector_observations
#print("State:",state)
print("State matrix size:", state.shape)

env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
agent.reset()
action = agent.act(state)
print("Action matrix size:", action.shape)
env_info = env.step(action)[brain_name]
#print("env_info:",env_info)

next_state = env_info.vector_observations
#print("Next state:",next_state)
#print("Next state size:",next_state.shape[1])
reward = env_info.rewards
reward = np.array(reward)
print("reward:",reward)
print("reward shape:", reward.shape)
done = env_info.local_done
print(np.zeros(num_agents) )

State matrix size: (20, 33)
Action matrix size: (20, 4)
reward: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
reward shape: (20,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
score_threshold=30.0

def ddpg(n_episodes=1000, max_t=1000, print_interval=20):
    
    scores_deque = deque(maxlen=100)
    scores = []         # initialize scores for all episodes
    max_score = -np.Inf
    
    time_elapsed = 0 # initialize the time
    
    ### start training ###
    for i_episode in range(1, n_episodes+1):
        time_beginning = time.time() # starting time for this episode
        # initialize the environment
        env_info = env.reset(train_mode=True)[brain_name]    # reset the environment
        states = env_info.vector_observations                # get the current state (for each agent)
        
        agent.reset()     # reset the agent
        episode_score = np.zeros(num_agents) # initialize the score in one episode 
        
        ###### start one episode training
        for t in range(max_t):
            actions = agent.act(states)                      # select an action (for each agent) using DDPG
            
            # take actions
            env_info = env.step(actions)[brain_name]        # send all actions to tne environment 
            next_states = env_info.vector_observations      # get next state (for each agent)
            rewards = env_info.rewards                      # get reward (for each agent
            dones = env_info.local_done                     # see if episode finished

            agent.step(states, actions, rewards, next_states, dones)
            #print("***Reward:", rewards) # test
            episode_score += np.array(rewards)
            states = next_states
            if np.any(dones):
                break
        ##### end of max_t (finish one episode)
        time_end = time.time() # ending time for this episode
        # calculate the scores
        mean_episode_score = np.mean(episode_score)    # take average scores for one episode
        
        scores_deque.append(mean_episode_score)
        average_score = np.mean(scores_deque) # average scores
        
        scores.append(mean_episode_score) # attach episode score to scores (for all episode)
        
        # test
        #print("\n1. mean_episode_score:", mean_episode_score)
        #print("\n2. scores_deque:", scores_deque)
        #print("\n3. scores.append:", scores)
        #print("\n4. average_score:", average_score)
        time_elapsed = time_elapsed + (time_end - time_beginning) # how much time it takes for one episodes

        # save the weights
        print('\rEpisode {}\tAverage Score: {:.2f}\tTime: {:.2f} s:'.format(i_episode, average_score, time_elapsed), end="")
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        
        # print average scores
        if i_episode % print_interval == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}\tTime: {:.2f} s:'.format(i_episode, average_score, time_elapsed))
        
        # stop the training the score is more than required
        if average_score >= score_threshold:
            print('\nEnvironment solved in {} episodes!\tAverage Score: {:.2f}'.format(i_episode-window_size, average_score))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            break
            
        # stop the training if it doesn't work
        if i_episode > 30 and average_score < 0.5:
            print("\nThe training has stopped because it is not working. Please redesign the algorithm")
            break
        #if i_episode > 100 and average_score <1:
        #    print("\nThe training has stopped because it is not working well. Please redesign the algorithm")
        #    break
            
    ### end of training (finished all episodes)

        
    np.save('scores.npy', scores)
    return scores


In [ ]:
env # call env window

In [10]:
scores = ddpg()

Episode 20	Average Score: 4.67	Time: 1987.07 s:
Episode 40	Average Score: 10.20	Time: 4390.73 s:
Episode 60	Average Score: 12.75	Time: 7317.22 s:
Episode 80	Average Score: 15.07	Time: 10233.46 s:
Episode 100	Average Score: 16.32	Time: 12985.38 s:
Episode 120	Average Score: 19.85	Time: 15833.32 s:
Episode 140	Average Score: 21.41	Time: 18646.27 s:
Episode 160	Average Score: 22.42	Time: 21436.53 s:
Episode 180	Average Score: 22.43	Time: 24206.64 s:
Episode 200	Average Score: 22.17	Time: 26979.41 s:
Episode 220	Average Score: 21.55	Time: 29796.12 s:
Episode 240	Average Score: 20.22	Time: 32569.96 s:
Episode 260	Average Score: 18.56	Time: 35408.92 s:
Episode 280	Average Score: 16.83	Time: 38267.52 s:
Episode 300	Average Score: 15.20	Time: 41206.41 s:
Episode 320	Average Score: 13.31	Time: 44117.85 s:
Episode 340	Average Score: 11.61	Time: 47021.30 s:
Episode 360	Average Score: 10.57	Time: 49867.82 s:
Episode 366	Average Score: 10.36	Time: 50713.91 s:

KeyboardInterrupt: 

In [ ]:
# plot the reward figures
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 4. Run the trained agent

In [ ]:
# load the weights to the agent
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)

while True:
    actions = agent.act(states)       # select an action (for each agent)
    
    # take action
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

### 5. Close the environment

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 